### Creating classes

In [196]:
from collections import defaultdict
from math import sqrt
import numpy as np
from scipy.stats import norm
from tqdm import tqdm

In [603]:
class Derivative:
    """ Enabling the usage of +, *, -, etc. """
    def __add__(self, other):
        return Add(self, other)
    
    def __radd__(self, other):
        return rAdd(self, other)
    
    def __sub__(self, other):
        return Sub(self, other)

    def __rsub__(self, other):
        return rSub(self, other)
    
    def __mul__(self, other):
        return Mul(self, other)
    
    def __rmul__(self, other):
        return rMul(self, other)
    
    def __truediv__(self, other):
        return Div(self, other)

    def __rtruediv__(self, other):
        return rDiv(self, other)
   
    def Sin(self):
        return Sin(self)
    
    def __pow__(self, power):
        return Pow(self, power)
    
    def Exp(self):
        return Exp(self)
    
    def Cdf(self):
        return Cdf(self)
    
    def __ge__(self, other):
        return Ge(self, other)
    
    def __le__(self, other):
        return Le(self, other)
    
    def __hash__(self):                 # we need this for creating the dictionary with keys as Derivatives object
        return hash(str(self))

Leaf nodes

In [604]:
class Var(Derivative):
    """ A leaf node (a node which doesn't have any child) """
    
    def __init__(self, value):
        self.value = value      # the scalar value of the node

Adding the nodes

In [605]:
class Add(Derivative):
    """ The node that results from adding two nodes """
    def __init__(self, node_a, node_b):
        if isinstance(node_a, (int, float, np.ndarray)): 
            self.value = node_a + node_b.value
            self.grad = [(node_b, 1)]
        elif isinstance(node_b, (int, float, np.ndarray)): 
            self.value = node_a.value + node_b
            self.grad = [(node_a, 1)]
        else:      
            self.value = node_a.value + node_b.value    # value of the node
            self.grad = [(node_a, 1), (node_b, 1)]      # partial derivatives of nodes - value 1 for derivative in respect to node_a and 1 for node_b

In [606]:
class rAdd(Derivative):
    """ The node that results from adding two nodes """
    def __init__(self, node_a, node_b):
        if isinstance(node_b, (int, float)): 
            self.value = node_b + node_a.value
            self.grad = [(node_a, 1)]
        elif isinstance(node_a, (int, float)): 
            self.value = node_b.value + node_a
            self.grad = [(node_b, 1)]
        else:      
            self.value = node_b.value + node_a.value    # value of the node
            self.grad = [(node_b, 1), (node_a, 1)]      # partial derivatives of nodes - value 1 for derivative in respect to node_a and 1 for node_b

Substract the nodes

In [607]:
class Sub(Derivative):
    """ The node that results from subtracting two nodes """
    def __init__(self, node_a, node_b):
        if isinstance(node_a, (int, float)): 
            self.value = node_a - node_b.value
            self.grad = [(node_b, -1)]
        elif isinstance(node_b, (int, float)): 
            self.value = node_a.value - node_b
            self.grad = [(node_a, 1)]
        else:      
            self.value = node_a.value - node_b.value    # value of the node
            self.grad = [(node_a, 1), (node_b, -1)]     # partial derivatives of nodes - should have again the structure as [(node_a, value), (node_b, value)]

In [608]:
class rSub(Derivative):
    """ The node that results from subtracting two nodes """
    def __init__(self, node_a, node_b):
        if isinstance(node_b, (int, float)): 
            self.value = node_b - node_a.value
            self.grad = [(node_a, -1)]
        elif isinstance(node_a, (int, float)): 
            self.value = node_b.value - node_a
            self.grad = [(node_b, 1)]
        else:      
            self.value = node_b.value - node_a.value    # value of the node
            self.grad = [(node_b, 1), (node_a, -1)]     # partial derivatives of nodes - should have again the structure as [(node_a, value), (node_b, value)]

Multiplication of nodes

In [609]:
class Mul(Derivative):
    """ The node that results from multiplying two nodes """
    def __init__(self, node_a, node_b):
        if isinstance(node_a, (int, float, np.ndarray)): 
            self.value = node_a * node_b.value
            self.grad = [(node_b, node_a)]
        elif isinstance(node_b, (int, float, np.ndarray)): 
            self.value = node_a.value * node_b
            self.grad = [(node_a, node_b)]
        else:      
            self.value = node_a.value * node_b.value
            self.grad = [(node_a, node_b.value), (node_b, node_a.value)]        # f = x*y   df / dx = y   df / dy = x

In [610]:
class rMul(Derivative):
    """ The node that results from multiplying two nodes """
    def __init__(self, node_a, node_b):
        if isinstance(node_b, (int, float, np.ndarray)): 
            self.value = node_b * node_a.value
            self.grad = [(node_a, node_b)]
        elif isinstance(node_a, (int, float, np.ndarray)): 
            self.value = node_b.value * node_a
            self.grad = [(node_b, node_a)]
        else:      
            self.value = node_b.value * node_a.value
            self.grad = [(node_b, node_a.value), (node_a, node_b.value)]                                # f = x*y   df / dx = y   df / dy = x

Dividing nodes

In [611]:
class Div(Derivative):
    """ The node that results from dividing one node by another """
    def __init__(self, node_a, node_b):
        if isinstance(node_a, (int, float)): 
            self.value = node_a / node_b.value
            self.grad = [(node_b, -node_a/(node_b.value**2))]
        elif isinstance(node_b, (int, float)): 
            self.value = node_a.value / node_b
            self.grad = [(node_a, 1/node_b)]
        else:      
            self.value = node_a.value / node_b.value
            self.grad = [(node_a, 1/node_b.value), (node_b, -node_a.value/(node_b.value**2))]           # f = x/y   df / dx = 1/y    df / dy = -x / (y^2)

In [612]:
class rDiv(Derivative):
    """ The node that results from dividing one node by another """
    def __init__(self, node_a, node_b):
        if isinstance(node_b, (int, float)): 
            self.value = node_b / node_a.value
            self.grad = [(node_a, -node_b/(node_a.value**2))]
        elif isinstance(node_a, (int, float)): 
            self.value = node_b.value / node_a
            self.grad = [(node_b, 1/node_a)]
        else:      
            self.value = node_b.value / node_a.value
            self.grad = [(node_b, 1/node_a.value), (node_a, -node_b.value/(node_a.value**2))]           # f = x/y   df / dx = 1/y    df / dy = -x / (y^2)

Logarithm of one mode

In [613]:
class Log(Derivative):
    """ The node that results from sin(node) """
    
    def __init__(self, node):
        if isinstance(node, (int, float)): 
            self.value = np.log(node) 
            self.grad = [(node, 0)]
        else:
            self.value = np.log(node.value)                          
            self.grad = [(node, 1/node.value)]                 

Sinus of one node

In [614]:
class Sin(Derivative):
    """ The node that results from sin(node) """
    
    def __init__(self, node):
        if isinstance(node, (int, float)): 
            self.value = np.sin(node) 
            self.grad = [(node, 0)]
        else:
            self.value = np.sin(node.value)                          # use np.sin() function
            self.grad = [(node, np.cos(node.value))]                 # only one derivative, since it takes only one node - use np.cos() function

Cosinus of one node

In [615]:
class Cos(Derivative):
    """ The node that results from cos(node) """
    
    def __init__(self, node):
        if isinstance(node, (int, float)): 
            self.value = np.cos(node)
            self.grad = [(node, 0)]
        else:
            self.value = np.cos(node.value)                          
            self.grad = [(node, -np.sin(node.value))]                

Powers

In [616]:
class Pow(Derivative):
    """ The node that results as node ^ power """
    
    def __init__(self, node, power):
        self.value = node.value**power                                         # reminder of power operation in python: **
        self.grad = [(node, power*node.value**(power-1))]                      # (one derivative)    f = x^n   df/dx = n * x ^ (n-1)

Exponential

In [617]:
class Exp(Derivative):
    """ The node that results from exp(node) """

    def __init__(self, node):
        if isinstance(node, (int, float)): 
            self.value = np.exp(node)  
            self.grad = [(node, 0)]
        else:
            self.value = np.exp(node.value)                         
            self.grad = [(node, np.exp(node.value))]   

Squareroot

In [618]:
class Sqrt(Derivative):
    """ The node that results from sqrt(node) """
    
    def __init__(self, node):
        if isinstance(node, (int, float)): 
            self.value = np.sqrt(node)
            self.grad = [(node, 0)]
        else:
            self.value = np.sqrt(node.value)                  
            self.grad = [(node, 1/(2*np.sqrt(node.value)))]  

$ a^x $

In [619]:
class Pot(Derivative):
    """ The node that results from a ^ node """
    
    def __init__(self, a, node):
        self.value = a**node.value                          
        self.grad = [(node, a**node.value * np.log(a))]                

Cdf

In [620]:
class Cdf(Derivative):
    """ The node that results from cdf(node) """

    def __init__(self, node):
        if isinstance(node, (int, float)): 
            self.value = norm.cdf(node) 
            self.grad = [(node, norm.pdf(node) )]
        else:
            self.value = norm.cdf(node.value)            
            self.grad = [(node, norm.pdf(node.value))]

Overskrivning af >= og <=

In [621]:
class Ge(Derivative):
    """ >= """
            
    def __init__(self, node_a, node_b):
        if isinstance(node_a, (int, float)): 
            self.value = node_a >= node_b.value
        elif isinstance(node_b, (int, float)): 
            self.value = node_a.value >= node_b
        else:      
            self.value = node_a.value >= node_b.value

In [622]:
class Le(Derivative):
    """ <= """

    def __init__(self, node_a, node_b):
        if isinstance(node_a, (int, float)): 
            self.value = node_a <= node_b.value
        elif isinstance(node_b, (int, float)): 
            self.value = node_a.value <= node_b
        else:      
            self.value = node_a.value <= node_b.value

### Create a function for getting the gradients values

In [623]:
def Get_Gradient(parent_node):
    """ Go down the graph, and compute derivative of `parent_node` with respect to each node """
    
    # we will create a dictionary 'gradient' which will have the nodes as keys and its derivatives as values
    gradients = defaultdict(lambda: 0)    # initialize the dictionary so when calling a non-existing key the value of 0 is assigned
    
    # stack will represent the list of tuples (node, node_derivative) 
    stack = parent_node.grad.copy()     
    
    while stack:                             # loop for each different branch
        # get node and node_derivative from the top of the stack - function pop()
        temp = stack.pop()                   
        node = temp[0]
        node_derivative = temp[1]            
        # add to the value of derivative of the node (gradients[node]) value node_derivative
        gradients[node] = gradients[node] + node_derivative  
        
        if not isinstance(node, Var):        # if the node has children, put them onto the stack
            # loop for each node in one branch
            for child_node, child_node_derivative in node.grad:                   
                # append child_node and child_node_derivative * node_derivative to the stack
                stack.append((child_node, child_node_derivative * node_derivative))
                
    return dict(gradients)

### Test: Black-Scholes (European Option)

In [624]:
import numpy as np
import random
import matplotlib.pyplot as plt
from matplotlib import style 
style.use('ggplot')
from scipy.stats import norm
import scipy

In [625]:
def euro_call(S0, K, T, r, sigma):
    d1 = (Log(S0/K)+(r+1/2*(sigma**2))*T)/(sigma*Sqrt(T))
    d2 = d1 - sigma*Sqrt(T)
    return S0 * Cdf(d1)  - K * Exp((-1)*r * T) * Cdf(d2)

In [626]:
S0_val = 100
K_val = 100
T_val = 1
r_val = 0.07
sigma_val = 0.2
n_simulations = 10000
n_steps = 1
S0 = Var(S0_val)
K = Var(K_val)
T = Var(T_val)
r = Var(r_val)
sigma = Var(sigma_val)

In [627]:
y = euro_call(S0, K, T, r, sigma)
gradients = Get_Gradient(y)

print('Value of f equals', y.value)
print('The partial derivative of y with respect to S0 =', gradients[S0])
print('The partial derivative of y with respect to K =', gradients[K])
print('The partial derivative of y with respect to T =', gradients[T])
print('The partial derivative of y with respect to r =', gradients[r])
print('The partial derivative of y with respect to sigma =', gradients[sigma])

Value of f equals 11.541470170672412
The partial derivative of y with respect to S0 = 0.6736447797120796
The partial derivative of y with respect to K = -0.5582300780053555
The partial derivative of y with respect to T = 7.512880170653974
The partial derivative of y with respect to r = 55.82300780053558
The partial derivative of y with respect to sigma = 36.05269624616482


### Pricing & Adjoints (European Option) 

In [579]:
def European_Option_Price_Adjoints(S0, K, T, r, sigma, n_simulations, n_steps):
    timeline = np.arange(0, T.value, 1/n_steps)
    np.random.seed(123)
    Z = np.random.normal (0, 1, size = (n_simulations , len(timeline)))
    
    def European_Option_Price(S0, K, T, r, sigma, Z):
        dt = T/n_steps
        d1 = r - (sigma**2) / 2
        d2 = d1 * dt + sigma * Sqrt(dt) * Z 
        S0 = S0 * Exp(d2)
        if ((S0 - K).value >= 0) == True:
            return (S0 - K) * Exp((-1) * r * T)
        else:
            return (S0 - K) * 0 * Exp((-1) * r * T)
    
    price = []
    adjoints = defaultdict(int)
    
    for i in tqdm(Z[:,-1]):
        y = European_Option_Price(S0, K, T, r, sigma, i)
        gradients = Get_Gradient(y)
        price.append(y.value)
        adjoints["S_"] += gradients[S0]/n_simulations
        adjoints["K_"] += gradients[K]/n_simulations
        adjoints["T_"] += gradients[T]/n_simulations
        adjoints["r_"] += gradients[r]/n_simulations
        adjoints["sigma_"] += gradients[sigma]/n_simulations
    
    price = np.mean(price)
    
    return price, adjoints

In [580]:
European_Option = European_Option_Price_Adjoints(S0, K, T, r, sigma, n_simulations, n_steps)

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 7396.23it/s]


In [581]:
European_Price = European_Option[0]
European_Price

11.679930554682997

In [582]:
European_Adjoints = European_Option[1]
European_Adjoints

defaultdict(int,
            {'S_': 0.6808043272079407,
             'K_': -0.5640050216610463,
             'T_': 7.600638310393247,
             'r_': 56.400502166113164,
             'sigma_': 36.52603158765489})

In [422]:
# S0_val = 100
# K_val = 100
# T_val = 1
# r_val = 0.07
# sigma_val = 0.2
# n_simulations = 1000
# n_steps = 1
# S0 = Var(S0_val)
# K = Var(K_val)
# T = Var(T_val)
# r = Var(r_val)
# sigma = Var(sigma_val)

In [434]:
# ### TEST MED REDUCE()

# def European_Option_Price_Adjoints_(S0, K, T, r, sigma, n_simulations, n_steps):
#     timeline = np.arange(0, T.value, 1/n_steps)
#     np.random.seed(123)
#     Z = np.random.normal(0, 1, size = (n_simulations , len(timeline)))
    
#     def European_Option_Price(S0, K, T, r, sigma, Z, j):
#         a = Z
#         b = np.ones((n_simulations, n_steps+1))*S0
#         b[:,1:] = a
#         Z = b
#         dt = T/n_steps
#         d1 = r - (sigma**2) / 2
#         S0 = reduce(lambda F, i: F * Exp(d1 * dt + sigma * Sqrt(dt) * i), Z[j,:])
#         if ((S0 - K).value >= 0) == True:
#             return (S0 - K) * Exp((-1) * r * T)
#         else:
#             return (S0 - K) * 0 * Exp((-1) * r * T)
    
#     price = []
#     adjoints = defaultdict(int)
    
#     for i in tqdm(range(0, n_simulations)):
#         y = European_Option_Price(S0, K, T, r, sigma, Z, i)
#         gradients = Get_Gradient(y)
#         price.append(y.value)
#         adjoints["S_"] += gradients[S0]/n_simulations
#         adjoints["K_"] += gradients[K]/n_simulations
#         adjoints["T_"] += gradients[T]/n_simulations
#         adjoints["r_"] += gradients[r]/n_simulations
#         adjoints["sigma_"] += gradients[sigma]/n_simulations
    
#     price = np.mean(price)
    
#     return price, adjoints

### Test: Black-Scholes (Asian Option)

In [174]:
def BS_Price_Asian_Options(S0, K, T, r, sigma):
    sqrt3 = Sqrt(3).value
    
    b = (1/2) * (r - (1/2) * ((sigma / sqrt3)**2))
    
    d1 = (Log(S0 / K) + (b + (1/2) * ((sigma / sqrt3)**2) ) * T)/ ((sigma / sqrt3) * Sqrt(T))
    
    d2 = d1 - (sigma / sqrt3)*Sqrt(T)
    
    return S0 * Exp((b - r) * T) * Cdf(d1) - K * Exp(-(1) * r * T) * Cdf(d2)

In [175]:
S0_val = 100
K_val = 100
T_val = 1
r_val = 0.07
sigma_val = 0.2

In [414]:
S0 = Var(S0_val)
K = Var(K_val)
T = Var(T_val)
r = Var(r_val)
sigma = Var(sigma_val)

y = BS_Price_Asian_Options(S0, K, T, r, sigma)

gradients = Get_Gradient(y)

print('Value of f equals', y.value)
print('The partial derivative of y with respect to S0 =', gradients[S0])
print('The partial derivative of y with respect to K =', gradients[K])
print('The partial derivative of y with respect to T =', gradients[T])
print('The partial derivative of y with respect to r =', gradients[r])
print('The partial derivative of y with respect to sigma =', gradients[sigma])

Value of f equals 6.024544283554853
The partial derivative of y with respect to S0 = 0.6063517314661314
The partial derivative of y with respect to K = -0.546106288630583
The partial derivative of y with respect to T = 3.596224538143864
The partial derivative of y with respect to r = 24.29304228975174
The partial derivative of y with respect to sigma = 18.95711577861243


### Pricing & Adjoints (Asian Option) 

In [441]:
K_val = 100
T_val = 1
r_val = 0.07
n_simulations = 1000
n_steps = 252
S0_val = 100
sigma_val = 0.2

K = Var(K_val)
T = Var(T_val)
r = Var(r_val)
S0 = Var(S0_val)
sigma = Var(sigma_val)

In [442]:
def Asian_Option_Price_Adjoints_(S0, K, T, r, sigma, n_simulations, n_steps):
    timeline = np.arange(0, 1, 1/n_steps)      
    Z = np.random.normal (0, 1, size = (n_simulations, len(timeline)))
    
    def Asian_Option_Price(S0, K, T, r, sigma, Z, j):
        dt = T/n_steps
        d1 = r - (sigma**2) / 2
        F = S0
        temp = 0
        p = 0
    
        for i in range(0, n_steps):
            p += F
            d2 = d1 * dt + sigma * Sqrt(dt) * Z[j,i]
            temp = F * Exp(d2)
            F = temp
        S_T = p / n_steps
   
        if ((S_T - K).value >= 0) == True: 
            return (S_T - K) * Exp((-1) * r * T)
        else:
            return (S_T - K) * 0 * Exp((-1) * r * T)
   
    price = []
    adjoints = defaultdict(int)

    for j in tqdm(range(0, n_simulations)):
        y = Asian_Option_Price(S0, K, T, r, sigma, Z, j)
        price.append(y.value)
        gradients = Get_Gradient(y)
        adjoints["S_"] += gradients[S0]/n_simulations
        adjoints["K_"] += gradients[K]/n_simulations
        adjoints["T_"] += gradients[T]/n_simulations
        adjoints["r_"] += gradients[r]/n_simulations
        adjoints["sigma_"] += gradients[sigma]/n_simulations
    
    price = np.mean(price)
    
    return price, adjoints

In [443]:
Asian_Option = Asian_Option_Price_Adjoints_(S0, K, T, r, sigma, n_simulations, n_steps)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [24:17<00:00,  1.46s/it]


In [444]:
Asian_Price = Asian_Option[0]
Asian_Price

6.815025861207657

In [445]:
Asian_Adjoints = Asian_Option[1]
Asian_Adjoints

defaultdict(int,
            {'S_': 0.6303837320153634,
             'K_': -0.5622334734032888,
             'T_': 4.168203541842798,
             'r_': 25.548648305612094,
             'sigma_': 23.79798160449956})

In [ ]:
# from functools import reduce
# import operator

# def prod(node_a, node_b):
#     if isinstance(node_a, (int, float)) andAsian_Option = Asian_Option_Price_Adjoints_(S0, K, T, r, sigma, n_simulations, n_steps)Asian_Option = Asian_Option_Price_Adjoints_(S0, K, T, r, sigma, n_simulations, n_steps)Asian_Adjoints = Asian_Option[1]
Asian_Adjoints isinstance(node_b, (int, float)): 
#         return reduce(operator.mul, range(node_a, node_b), 1)
#     elif isinstance(node_b, (int, float)): 
#         return reduce(operator.mul, range(node_a.value, node_b), 1)
#     elif isinstance(node_a, (int, float)): 
#         return reduce(operator.mul, range(node_a, node_b.value), 1)
#     else: 
#         return reduce(operator.mul, range(node_a.value, node_b.value), 1)

In [330]:
# ### TEST MED REDUCE()

# def Asian_Option_Price_Adjoints_(S0, K, T, r, sigma, n_simulations, n_steps):
#     timeline = np.arange(0, 1, 1/n_steps)      
#     Z = np.random.normal (0 , 1 , size = (n_simulations , len(timeline)))
    
#     def Asian_Option_Price(S0, K, T, r, sigma, Z, j):
#         dt = T/n_steps
#         d1 = r - (sigma**2) / 2
#         F = S0
#         temp = 0
#         p = 0
        
#         temp0 = reduce(lambda F, i: F * Exp(d1 * dt + sigma * Sqrt(dt) * i), Z[j,:])
        
# #         for i in range(0, n_steps):
# #             p += F
# #             d2 = d1 * dt + sigma * Sqrt(dt) * Z[j,i]
# #             temp = F * Exp(d2)
# #             F = temp

#         print(temp0.value)
        
#         temp0 = temp0*S0
#         print("ny:", temp0.value)
    
#         S_T = temp0**(1/n_steps)
   
#         if ((S_T - K).value >= 0) == True: 
#             return (S_T - K) * Exp((-1) * r * T)
#         else:
#             return (S_T - K) * 0 * Exp((-1) * r * T)
   
#     price = []
#     adjoints = defaultdict(int)

#     for j in tqdm(range(0, n_simulations)):
#         y = Asian_Option_Price(S0, K, T, r, sigma, Z, j)
#         price.append(y.value)
# #         gradients = Get_Gradient(y)
# #         adjoints["S_"] += gradients[S0]/n_simulations
# #         adjoints["K_"] += gradients[K]/n_simulations
# #         adjoints["T_"] += gradients[T]/n_simulations
# #         adjoints["r_"] += gradients[r]/n_simulations
# #         adjoints["sigma_"] += gradients[sigma]/n_simulations
    
#     price = np.mean(price)
    
#     return price, adjoints

### CVA Implementation

In [680]:
    def CVA_Calculation(R, price, spread, T, n_steps):
        t_i = 1/n_steps
        lamb = spread/(1-R)
        Q = 0
        for i in np.arange(0, T.value+t_i, t_i): 
            q = 1 - Exp((-1)* lamb * i)
            Q += q
        CVA = (1-R) * price * Q
        return CVA

# Vi definerer, at q[i] har dimensionen T*n_steps. 
# Vi antager, at q[i] defineres per år, jf. fx Moody's kreditratings, dvs. n_steps=1.

# t_i = T*1/n_steps
# Input er kreditspænd, spread. Vi antager, at spread og dvs. lambda er konstant. 
# Hazard rate er l = spread/(1-R).
# Da er q[t] = 1-exp(-l*t_i).


In [681]:
def European_Option_Price_Adjoints_CVA(S0, K, T, r, sigma, n_simulations, n_steps, R, spread):
    timeline = np.arange(0, T.value, 1/n_steps)
    np.random.seed(123)
    Z = np.random.normal (0, 1, size = (n_simulations , len(timeline)))
    
    def European_Option_Price(S0, K, T, r, sigma, Z):
        dt = T/n_steps
        d1 = r - (sigma**2) / 2
        d2 = d1 * dt + sigma * Sqrt(dt) * Z 
        S0 = S0 * Exp(d2)
        if ((S0 - K).value >= 0) == True:
            return (S0 - K) * Exp((-1) * r * T)
        else:
            return (S0 - K) * 0 * Exp((-1) * r * T)
    
    def CVA_Calculation(R, price, spread, T, n_steps):
        t_i = 1/n_steps
        lamb = spread/(1-R)
        Q = 0
        for i in np.arange(0, T.value + t_i, t_i): 
            q = 1 - Exp((-1) * lamb * i)
            Q += q
        CVA = (1-R) * price * Q
        return CVA
    
    price = []
    CVA = []
    adjoints = defaultdict(int)
    
    for i in tqdm(Z[:,-1]):
        y = European_Option_Price(S0, K, T, r, sigma, i)
        price.append(y.value)
        z = CVA_Calculation(R, y, spread, T, n_steps)
        CVA.append(z.value)
        gradients = Get_Gradient(z)
        adjoints["R_"] += gradients[R]/n_simulations
        adjoints["spread_"] += gradients[spread]/n_simulations
        adjoints["S_"] += gradients[S0]/n_simulations
        adjoints["K_"] += gradients[K]/n_simulations
        adjoints["T_"] += gradients[T]/n_simulations
        adjoints["r_"] += gradients[r]/n_simulations
        adjoints["sigma_"] += gradients[sigma]/n_simulations
    
    price = np.mean(price)
    CVA = np.mean(CVA)
    
    return price, CVA, adjoints

In [697]:
S0_val = 100
K_val = 100
T_val = 1
r_val = 0.07
sigma_val = 0.2
n_simulations = 100000
n_steps = 1

R_val = 0.40
spread_val = 0.02

S0 = Var(S0_val)
K = Var(K_val)
T = Var(T_val)
r = Var(r_val)
sigma = Var(sigma_val)

R = Var(R_val)
spread = Var(spread_val)

In [704]:
European_Option = European_Option_Price_Adjoints_CVA(S0, K, T, r, sigma, n_simulations, n_steps, R, spread)

100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [00:25<00:00, 3862.68it/s]


In [705]:
European_Price = European_Option[0]
European_Price

11.563321726153035

In [706]:
European_CVA = European_Option[1]
European_CVA

0.23714567305942622

In [707]:
European_Adjoints = European_Option[2]
European_Adjoints

defaultdict(int,
            {'R_': -0.014945112791514443,
             'spread_': 12.156185908801568,
             'S_': 0.013821675217155696,
             'K_': -0.011450218486562073,
             'T_': 0.15432477129479452,
             'r_': 1.1450218486567054,
             'sigma_': 0.741732418888659})

In [728]:
TEST_European_Option = European_Option_Price_Adjoints_CVA(S0, K, T, r, sigma, n_simulations, n_steps, R, spread+1)

100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [00:27<00:00, 3641.73it/s]


In [729]:
TEST_European_CVA = TEST_European_Option[1]
TEST_European_CVA - 0.23714567305942622

54.37459020414506

In [727]:
(CVA_Calculation(R, European_Price, spread+0.00000000001, T, n_steps).value - CVA_Calculation(R, European_Price, spread, T, n_steps).value)/0.00000000001

12.156126105722365